In [60]:
import pandas #ipython notebook
titanic = pandas.read_csv("titanic_train.csv")
titanic.head(3)
#print(titanic.shape)
#print (titanic.describe())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [61]:
titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())#缺失值以age的平均值填充
print(titanic.describe())
titanic["Age"].median()

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.361582    0.523008   
std     257.353842    0.486592    0.836071   13.019697    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   22.000000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   35.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


28.0

In [62]:
#使用unique去掉重复值
print(titanic["Sex"].unique())
# Replace all the occurences of male with the number 0.
#以编码01取代性别字符
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female", "Sex"] = 1


['male' 'female']


In [63]:
print(titanic["Embarked"].unique())
titanic["Embarked"]
titanic["Embarked"] = titanic["Embarked"].fillna('S')
titanic.loc[titanic["Embarked"] == "S","Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2
titanic.loc[:,"Embarked"]

['S' 'C' 'Q' nan]


0      0
1      1
2      0
3      0
4      0
5      2
6      0
7      0
8      0
9      1
10     0
11     0
12     0
13     0
14     0
15     0
16     2
17     0
18     0
19     1
20     0
21     0
22     2
23     0
24     0
25     0
26     1
27     0
28     2
29     0
      ..
861    0
862    0
863    0
864    0
865    0
866    1
867    0
868    0
869    0
870    0
871    0
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    0
881    0
882    0
883    0
884    0
885    2
886    0
887    0
888    0
889    1
890    2
Name: Embarked, Length: 891, dtype: int64

In [64]:
# Import the linear regression class
#线性回归模型
from sklearn.linear_model import LinearRegression
# Sklearn also has a helper that makes it easy to do cross validation
#交叉验证模型
from sklearn.cross_validation import KFold

# The columns we'll use to predict the target
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm class
alg = LinearRegression()
# Generate cross validation folds for the titanic dataset.  It return the row indices corresponding to train and test.
# We set random_state to ensure we get the same splits every time we run this.
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)
print(kf)
#titanic.shape[0]返回titanic行数，即样本数量m，对m个样本对交叉验证，n_folds做3层交叉验证，random_state=1随机种子数值为1
predictions = []
#print(train)
#print(test)
#print(kf)
for train, test in kf:
    # The predictors we're using the train the algorithm.  Note how we only take the rows in the train folds.
    train_predictors = (titanic[predictors].iloc[train,:])
    #titanic[predictors].iloc[train,:]
    # The target we're using to train the algorithm.
    train_target = titanic["Survived"].iloc[train]
    # Training the algorithm using the predictors and target.
    alg.fit(train_predictors, train_target)
    #把训练集和目标集放到线性回归模型里训练
    # We can now make predictions on the test fold
    test_predictions = alg.predict(titanic[predictors].iloc[test,:])
    #把测试集放到训练好的线性回归模型里进行测试
    predictions.append(test_predictions)

sklearn.cross_validation.KFold(n=891, n_folds=3, shuffle=False, random_state=1)


In [65]:
import numpy as np

# The predictions are in three separate numpy arrays.  Concatenate them into one.  
# We concatenate them on axis 0, as they only have one axis.
#print(predictions)
predictions = np.concatenate(predictions, axis=0)

# Map predictions to outcomes (only possible outcomes are 1 and 0)
predictions[predictions > .5] = 1
predictions[predictions <=.5] = 0
accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
#print（accuracy）

In [66]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
# Initialize our algorithm
#使用逻辑回归模型
alg = LogisticRegression(random_state=1)
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=3)
#此使用说明见本人博客http://blog.sina.com.cn/s/blog_639a2ad70102xbix.html
#使用交叉验证并求各交叉验证结果平均值
# Take the mean of the scores (because we have one for each fold)
print(titanic[predictors])
print(scores.mean())

     Pclass  Sex   Age  SibSp  Parch      Fare  Embarked
0         3    0  22.0      1      0    7.2500         0
1         1    1  38.0      1      0   71.2833         1
2         3    1  26.0      0      0    7.9250         0
3         1    1  35.0      1      0   53.1000         0
4         3    0  35.0      0      0    8.0500         0
5         3    0  28.0      0      0    8.4583         2
6         1    0  54.0      0      0   51.8625         0
7         3    0   2.0      3      1   21.0750         0
8         3    1  27.0      0      2   11.1333         0
9         2    1  14.0      1      0   30.0708         1
10        3    1   4.0      1      1   16.7000         0
11        1    1  58.0      0      0   26.5500         0
12        3    0  20.0      0      0    8.0500         0
13        3    0  39.0      1      5   31.2750         0
14        3    1  14.0      0      0    7.8542         0
15        2    1  55.0      0      0   16.0000         0
16        3    0   2.0      4  

In [67]:
titanic_test = pandas.read_csv("test.csv")
titanic_test["Age"] = titanic_test["Age"].fillna(titanic["Age"].median())
titanic_test["Fare"] = titanic_test["Fare"].fillna(titanic_test["Fare"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0 
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")

titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2
#把属性字符串编码成数字便于模型处理

In [68]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

# Initialize our algorithm with the default paramters
# n_estimators is the number of trees we want to make
# min_samples_split is the minimum number of rows we need to make a split
# min_samples_leaf is the minimum number of samples we can have at the place where a tree branch ends (the bottom points of the tree)
alg = RandomForestClassifier(random_state=1, n_estimators=10, min_samples_split=2, min_samples_leaf=1)
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
kf = cross_validation.KFold(titanic.shape[0], n_folds=3, random_state=1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=kf)

# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

0.785634118967


In [70]:
alg = RandomForestClassifier(random_state=1, n_estimators=100, min_samples_split=4, min_samples_leaf=2)
# Compute the accuracy score for all the cross validation folds.  (much simpler than what we did before!)
#使用随机森林模型来预测样本。
kf = cross_validation.KFold(titanic.shape[0], 3, random_state=1)
scores = cross_validation.cross_val_score(alg, titanic[predictors], titanic["Survived"], cv=kf)
print(kf)
# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

sklearn.cross_validation.KFold(n=891, n_folds=3, shuffle=False, random_state=1)
0.814814814815


In [71]:
# Generating a familysize column
titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"]
#增加家庭成员数量牲特征，内容是晚辈数量加长辈数量。
print(titanic.head(1))
# The .apply method generates a new series
titanic["NameLength"] = titanic["Name"].apply(lambda x: len(x))
#添加NameLength列，该列的内容如下：Name的内容经过（lambda x: len(x)）运算后形成的新一列。lambda相当于内建一个函数。即求Name的内容长度。
print(titanic.head(1))

   PassengerId  Survived  Pclass                     Name  Sex   Age  SibSp  \
0            1         0       3  Braund, Mr. Owen Harris    0  22.0      1   

   Parch     Ticket  Fare Cabin  Embarked  FamilySize  
0      0  A/5 21171  7.25   NaN         0           1  
   PassengerId  Survived  Pclass                     Name  Sex   Age  SibSp  \
0            1         0       3  Braund, Mr. Owen Harris    0  22.0      1   

   Parch     Ticket  Fare Cabin  Embarked  FamilySize  NameLength  
0      0  A/5 21171  7.25   NaN         0           1          23  


In [101]:
import re
#正则表达式模块
# A function to get the title from a name.
def get_title(name):
    # Use a regular expression to search for a title.  Titles always consist of capital and lowercase letters, and end with a period.
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
#正则表达式表示查找符合格式的字符串，格式为空格+英文字母+小数点,[A-Za-z]表示匹配A到Z和a-z的单个字母，加号表示多次匹配
#即多个A到Z和a-z之间的字母
# Get all the titles and print how often each one occurs.

titles = titanic["Name"].apply(get_title)
#对Name列应用get_title运算规则，提取每个姓名的title
print(titles)#每一个姓名的title各叫什么
#print(pandas.value_counts(titles))
# Map each title to an integer.  Some titles are very rare, and are compressed into the same codes as other titles.
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2}
for k,v in title_mapping.items():
    titles[titles == k] = v
#k相当于"Mr"，v相当于1，如此类推。
# Verify that we converted everything.
print(pandas.value_counts(titles))

# Add in the title column.
titanic["Title"] = titles

0          Mr
1         Mrs
2        Miss
3         Mrs
4          Mr
5          Mr
6          Mr
7      Master
8         Mrs
9         Mrs
10       Miss
11       Miss
12         Mr
13         Mr
14       Miss
15        Mrs
16     Master
17         Mr
18        Mrs
19        Mrs
20         Mr
21         Mr
22       Miss
23         Mr
24       Miss
25        Mrs
26         Mr
27         Mr
28       Miss
29         Mr
        ...  
861        Mr
862       Mrs
863      Miss
864        Mr
865       Mrs
866      Miss
867        Mr
868        Mr
869    Master
870        Mr
871       Mrs
872        Mr
873        Mr
874       Mrs
875      Miss
876        Mr
877        Mr
878        Mr
879       Mrs
880       Mrs
881        Mr
882      Miss
883        Mr
884        Mr
885       Mrs
886       Rev
887      Miss
888      Miss
889        Mr
890        Mr
Name: Name, Length: 891, dtype: object
1     517
2     183
3     125
4      40
5       7
6       6
7       5
10      3
8       3
9       2
Name: N

In [35]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
import matplotlib.pyplot as plt
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "FamilySize", "Title", "NameLength"]

# Perform feature selection
selector = SelectKBest(f_classif, k=5)
selector.fit(titanic[predictors], titanic["Survived"])

# Get the raw p-values for each feature, and transform from p-values into scores
scores = -np.log10(selector.pvalues_)

# Plot the scores.  See how "Pclass", "Sex", "Title", and "Fare" are the best?
plt.bar(range(len(predictors)), scores)
plt.xticks(range(len(predictors)), predictors, rotation='vertical')
plt.show()

# Pick only the four best features.
predictors = ["Pclass", "Sex", "Fare", "Title"]

alg = RandomForestClassifier(random_state=1, n_estimators=50, min_samples_split=8, min_samples_leaf=4)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

# The algorithms we want to ensemble.
# We're using the more linear predictors for the logistic regression, and everything with the gradient boosting classifier.
algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Title",]],
    [LogisticRegression(random_state=1), ["Pclass", "Sex", "Fare", "FamilySize", "Title", "Age", "Embarked"]]
]

# Initialize the cross validation folds
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

predictions = []
for train, test in kf:
    train_target = titanic["Survived"].iloc[train]
    full_test_predictions = []
    # Make predictions for each algorithm on each fold
    for alg, predictors in algorithms:
        # Fit the algorithm on the training data.
        alg.fit(titanic[predictors].iloc[train,:], train_target)
        # Select and predict on the test fold.  
        # The .astype(float) is necessary to convert the dataframe to all floats and avoid an sklearn error.
        test_predictions = alg.predict_proba(titanic[predictors].iloc[test,:].astype(float))[:,1]
        full_test_predictions.append(test_predictions)
    # Use a simple ensembling scheme -- just average the predictions to get the final classification.
    test_predictions = (full_test_predictions[0] + full_test_predictions[1]) / 2
    # Any value over .5 is assumed to be a 1 prediction, and below .5 is a 0 prediction.
    test_predictions[test_predictions <= .5] = 0
    test_predictions[test_predictions > .5] = 1
    predictions.append(test_predictions)

# Put all the predictions together into one array.
predictions = np.concatenate(predictions, axis=0)

# Compute accuracy by comparing to the training data.
accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
print(accuracy)


0.279461279461


In [37]:
titles = titanic_test["Name"].apply(get_title)
# We're adding the Dona title to the mapping, because it's in the test set, but not the training set
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2, "Dona": 10}
for k,v in title_mapping.items():
    titles[titles == k] = v
titanic_test["Title"] = titles
# Check the counts of each unique title.
print(pandas.value_counts(titanic_test["Title"]))

# Now, we add the family size column.
titanic_test["FamilySize"] = titanic_test["SibSp"] + titanic_test["Parch"]



1     240
2      79
3      72
4      21
7       2
6       2
10      1
5       1
Name: Title, dtype: int64


In [18]:
predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Title"]

algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), predictors],
    [LogisticRegression(random_state=1), ["Pclass", "Sex", "Fare", "FamilySize", "Title", "Age", "Embarked"]]
]

full_predictions = []
for alg, predictors in algorithms:
    # Fit the algorithm using the full training data.
    alg.fit(titanic[predictors], titanic["Survived"])
    # Predict using the test dataset.  We have to convert all the columns to floats to avoid an error.
    predictions = alg.predict_proba(titanic_test[predictors].astype(float))[:,1]
    full_predictions.append(predictions)

# The gradient boosting classifier generates better predictions, so we weight it higher.
predictions = (full_predictions[0] * 3 + full_predictions[1]) / 4
predictions

array([ 0.11682912,  0.47835566,  0.12614824,  0.13098157,  0.52105874,
        0.1435209 ,  0.64085331,  0.18003152,  0.67801353,  0.12111118,
        0.12105181,  0.20902118,  0.91068381,  0.1089127 ,  0.89142102,
        0.87713474,  0.16349859,  0.13907791,  0.54103238,  0.55661006,
        0.22420875,  0.5372079 ,  0.90572223,  0.38890588,  0.88384752,
        0.10357315,  0.90909441,  0.13746454,  0.31046249,  0.12665718,
        0.11663767,  0.18274855,  0.55220994,  0.49648575,  0.42415297,
        0.14191051,  0.50973638,  0.52452209,  0.13270506,  0.28366691,
        0.11145281,  0.46618807,  0.09996501,  0.83420617,  0.89959119,
        0.14983417,  0.31593419,  0.13789623,  0.89104185,  0.54189565,
        0.35666363,  0.17718135,  0.8307195 ,  0.87995521,  0.1755907 ,
        0.13741805,  0.10667279,  0.1234385 ,  0.12099736,  0.91285169,
        0.13099159,  0.15341948,  0.12993967,  0.66573206,  0.66343836,
        0.87272604,  0.67238712,  0.288265  ,  0.35236574,  0.85